## To import Electricity price from 2010 to 2023, and extend daily prices with monthly mean prices.

In [68]:
import pandas as pd

elec_df_monthly = pd.read_csv('ELECTRICITY.csv').iloc[:,1:].sort_values(by=['Year','Month'])

elec_df_monthly['Date'] = pd.to_datetime(elec_df_monthly[['Year', 'Month']].assign(DAY=1))
elec_df = elec_df_monthly.explode('Date') # The explode() method converts each element of the specified column(s) into a row.

# Generate a complete range of daily dates for the year for imputation
start_date = elec_df['Date'].min() # represents the starting point of your data
end_date = elec_df['Date'].max() + pd.offsets.MonthEnd(1)  # finds the maximum (or latest) date and include the last month fully
full_date_range = pd.date_range(start=start_date, end=end_date, freq='D') # generates a fixed-frequency DatetimeIndex

# Merge the full date range with the monthly averages to fill in all days
df_full_date_range = pd.DataFrame(full_date_range, columns=['Date'])
elec_df = pd.merge(df_full_date_range, elec_df, on='Date', how='left')
elec_df_daily = elec_df.ffill(axis=0) # to fill the missing value based on last valid observation following index sequence
print(elec_df_daily.info)
print(elec_df_daily.isna().sum().sort_values())

<bound method DataFrame.info of            Date    Year  Month  Electricity
0    2010-01-01  2010.0    1.0        46.29
1    2010-01-02  2010.0    1.0        46.29
2    2010-01-03  2010.0    1.0        46.29
3    2010-01-04  2010.0    1.0        46.29
4    2010-01-05  2010.0    1.0        46.29
...         ...     ...    ...          ...
5108 2023-12-27  2023.0   12.0       112.54
5109 2023-12-28  2023.0   12.0       112.54
5110 2023-12-29  2023.0   12.0       112.54
5111 2023-12-30  2023.0   12.0       112.54
5112 2023-12-31  2023.0   12.0       112.54

[5113 rows x 4 columns]>
Date           0
Year           0
Month          0
Electricity    0
dtype: int64


## 